based on Tetris implementation in Python, (c) 2010 "Kevin Chabowski"<kevin@kch42.de>

https://gist.github.com/silvasur/565419/d9de6a84e7da000797ac681976442073045c74a4

In [ ]:
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision.datasets import mnist
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
NUM_CLASSES = 4 # turn left, turn right, move left, move right
LEARING_RATE = 0.001
COLUMNS = 8
ROWS = 16

input_size = COLUMNS * ROWS * 2

In [ ]:
HIDDEN_SIZE = 50

class Model2Linear(nn.Module):
    def __init__(self):
        super().__init__()
        self.h1 = nn.Linear(input_size, HIDDEN_SIZE)
        self.h2 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.h3 = nn.Linear(HIDDEN_SIZE, NUM_CLASSES)
    
    def forward(self, x):
        x = x.data.view(-1, input_size)
        x = self.h1(x)
        x = F.relu(x)
        x = self.h2(x)
        x = F.relu(x)
        x = self.h3(x)
        x = F.softmax(x, dim=1)
        return x

In [1]:
def rotate_clockwise(shape):
    return [[shape[y][x]
             for y in range(len(shape))]
            for x in range(len(shape[0]) - 1, -1, -1)]


def check_collision(board, shape, offset):
    off_x, off_y = offset
    for cy, row in enumerate(shape):
        for cx, cell in enumerate(row):
            try:
                if cell and board[cy + off_y][cx + off_x]:
                    return True
            except IndexError:
                return True
    return False


def remove_row(board, row):
    del board[row]
    return [[0 for i in range(config['cols'])]] + board


def join_matrixes(mat1, mat2, mat2_off):
    off_x, off_y = mat2_off
    for cy, row in enumerate(mat2):
        for cx, val in enumerate(row):
            mat1[cy + off_y - 1][cx + off_x] += val
    return mat1


def new_board():
    board = [[0 for x in range(config['cols'])]
             for y in range(config['rows'])]
    board += [[1 for x in range(config['cols'])]]
    return board

In [1]:
class TetrisState():
    def move_left(self):
        pass

    def move_right(self):
        pass

    def turn_left(self):
        pass

    def turn_right(self):
        pass

    def next_state(self):
        return TetrisState()
    
    def rewards(self):
        pass
    
    def input(self):
        pass


In [ ]:
def evalulate(model):
    model.eval()
    loss = 0.0
    for data, labels in validation_loader:
        predictions_per_class = model(data.cuda())
        _, highest_prediction_class = predictions_per_class.max(1)
        loss += F.nll_loss(predictions_per_class, labels.cuda())
    return loss/len(validation_loader)

def learn():
    model = Model1Linear().cuda()
    optimizer = Adam(params=model.parameters(), lr=LEARING_RATE)

    for epoch in range(1000):
        model.train()
        for data, labels in loader:
            predictions_per_class = model(data.cuda())
            highest_prediction, highest_prediction_class = predictions_per_class.max(1)

            # how good are we? compare output with the target classes
            loss = F.nll_loss(predictions_per_class, labels.cuda())

            model.zero_grad() # ???
            loss.backward() # backpropagate
            optimizer.step()
        
        validation_loss = evalulate(model)
        print(f'Epoch: {epoch}, Loss: {validation_loss.item()}')
        
    return model
%time model = learn()